### Web Scraper to generate hisotrical weather data

In [ ]:
import requests
import pprint
from pymongo import MongoClient
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import selenium

First, clean up my data in order to identify which dates to scrape weather on

Pulling historic weather data from Pheonix, AZ

In [ ]:
'''
key items:

date
<div _ngcontent-app-root-c6="" class="date ng-star-inserted">1</div>

Weather description for the day
class = <div _ngcontent-app-root-c6="" class="phrase ng-star-inserted">mostly sunny</div>

Hi
<span _ngcontent-app-root-c6="" class="hi">80°</span>

Low
<span _ngcontent-app-root-c6="" class="low">61°</span>

Rain (in inches)
<span _ngcontent-app-root-c12="" class="wu-value wu-value-to">0.21</span>

FOCUS HERE
Phoenix, AZ
https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-4 
- date will be the changing variable
'''


In [ ]:
'''
client = MongoClient('localhost', 27017)
db = client.weather
pages = db.main_pages
'''

### <center> Create a List of URL's to pull data from <center>

In [ ]:
# Seems like I'll need to set up a web scraper for each year and have the month values change

url_lst_2019 = []

def web_scraper_links():
    url = 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-1'
    for x in range(1,13):
        url_lst.append(f"https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-{x}")
        
        
web_scraper_links()

In [ ]:
url_lst_2019

In [ ]:
'''
def link_check():
    for x in url_lst:
        if x == "https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2011-8":
            return "that shtuff here"
'''

In [ ]:
# link_check()

In [ ]:
'''
def web_scraper(lst):
    for x in lst:
        r = requests.get(f"{x}")
        pages.insert_one({'html': r.content})
        soup = BeautifulSoup(r.text, 'html.parser')
'''

In [ ]:
# web_scraper(url_lst)